#1.   Download the dataset from your dataset URL.

In [45]:
# Install the Kaggle library
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [46]:
# Make a directory named “.kaggle”
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [47]:
# Copy the “kaggle.json” into this new directory
!cp kaggle.json ~/.kaggle/

In [48]:
# Allocate the required permission for this file.
!chmod 600 ~/.kaggle/kaggle.json

In [50]:
# Download the required file
! kaggle competitions download -c sentiment-analysis-of-covid-19-related-tweets

! kaggle datasets download gpreda/covid19-tweets

sentiment-analysis-of-covid-19-related-tweets.zip: Skipping, found more recently modified local copy (use --force to force download)


In [56]:
# Unzip all the downloaded files
!unzip sentiment-analysis-of-covid-19-related-tweets.zip
!unzip covid19-tweets.zip

Archive:  sentiment-analysis-of-covid-19-related-tweets.zip
replace training.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  covid19-tweets.zip
replace covid19_tweets.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


#2.   Read external sources for the dataset to understand the schema and data.

In [63]:
!ls -l

total 169148
-rw-r--r-- 1 root root 68713814 Aug 30  2020 covid19_tweets.csv
-rw-r--r-- 1 root root 74043392 Mar 23 04:39 covid19_tweets.db
-rw-r--r-- 1 root root 29305228 Mar 23 04:55 covid19-tweets.zip
-rw-r--r-- 1 root root       66 Mar 23 04:52 kaggle.json
drwxr-xr-x 1 root root     4096 Mar 21 22:54 sample_data
-rw-r--r-- 1 root root   339144 Mar 23 04:53 sentiment-analysis-of-covid-19-related-tweets.zip
-rw-r--r-- 1 root root   530787 May 10  2021 training.csv
-rw-r--r-- 1 root root   260187 May 10  2021 validation.csv


In [59]:
# Read covid19_tweets
!head -n 10 covid19_tweets.csv | column -t -s , 

user_name                                                                    user_location           user_description                                                             user_created                                                                                                                   user_followers       user_friends                                                                                                                                user_favourites      user_verified                                                                                                                            date                 text                                                                                                                                         hashtags                                                                                                                                      source                                                                  

In [62]:
# Read training
!head -n 10 training.csv | column -t -s , 

ID  Tweet                                                                                                               Labels
1   NO JOKE I WILL HOP ON A PLANE RN! (Well after COVID19 lol)                                                          0 10
2   BanMediaHouse whose is responsible for spreading Fake and communal Stories in this pandemic Corona situation.       6
3   Im waiting for someone to say to me that all this corona thing is just an April fools joke                          3 4
4   He is a liar. Proven day night. Time again. Lies when the truth will do. COVID19                                    6
5   "NEW: U.S. CoronaVirus death toll reaches 4                                                                         000 after nearly 900 new deaths were reported today (BNO News) Covid19 CoronaVirusOutbreak"           8
6   "Coronavirus impact Govt extends I-T deadlines related to Sections 80C                                               80D"                               

In [61]:
# Read validation
!head -n 10 validation.csv | column -t -s , 

ID    Tweet
5001  "Forgot to a math test and I was failing but my teacher opening it back up                                maybe corona aint so bad"
5002  Corona effected came from Delhi in Bihar lakhisarai district at Mahisona village his name is pappu Khan
5003  Make CORONA END NA
5004  Imagine if the coronavirus pandemic was a big April fools joke the government pulled on us .....
5005  Howdy Ana. Where can I get married during the coronavirus? Any update from the Gov?
5006  Coronavirus live blog: Allergist and immunologist Dr. Purvi Parikh answers your questions
5007  We won't b talking about corona but about hunger killing pple
5008  Fuck Corona man... I couldve gone on a vacation next week to the states if this shit doesnt exist
5009  "Female-founded companies                                                                                 as a result                 have less cash on hand to carry them through any kind of unexpected event   like COVID19."


#3.   Create a database to store the data, using either SQL or NoSQL.

In [1]:
!sudo apt-get install sqlite3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
sqlite3 is already the newest version (3.31.1-4ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [2]:
!touch covid19_tweets.db

In [3]:
%load_ext sql

In [4]:
%sql sqlite:///covid19_tweets.db

'Connected: @covid19_tweets.db'

In [21]:
# %%sql

# DROP TABLE training_tweets;

 * sqlite:///covid19_tweets.db
Done.


[]


4.   Insert the data into the database, using either SQL or NoSQL.
5.   Connect to the database using Python and retrieve the data from the tables, using either SQL or NoSQL.
6.   Save the data into a pandas dataframe or other format.

In [7]:
# Create a table to store the CSV data

%%sql
CREATE TABLE tweets (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_name VARCHAR(255),
    user_location VARCHAR(255),
    user_description TEXT,
    user_created DATETIME,
    user_followers INT,
    user_friends INT,
    user_favourites INT,
    user_verified BOOLEAN,
    date DATETIME,
    text TEXT,
    hashtags VARCHAR(255),
    source VARCHAR(255),
    is_retweet BOOLEAN);

 * sqlite:///covid19_tweets.db
Done.


[]

In [10]:
!sqlite3 -separator ',' covid19_tweets.db ".import covid19_tweets.csv tweets"


In [6]:
%%sql
.separator ','

.import covid19_tweets.csv tweets

 * sqlite:///covid19_tweets.db
(sqlite3.OperationalError) near ".": syntax error
[SQL: .separator ','

.import covid19_tweets.csv tweets]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [35]:
import sqlite3
import csv

# Connect to the database
conn = sqlite3.connect('covid19_tweets.db')

# Create a cursor object
cursor = conn.cursor()

In [26]:
# Create a table to store the CSV data
create_table_query = '''
    CREATE TABLE training_tweets (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Tweet TEXT,
    Labels VARCHAR(255)
);
'''
cursor.execute(create_table_query)

In [27]:

# Execute the SQL command to insert data from CSV file
with open('training.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # skip header row
    for row in reader:
        cursor.execute('INSERT INTO training_tweets (id, Tweet, Labels) VALUES (?, ?, ?)', row)

# Commit the changes
conn.commit()

In [36]:
# Filtering the Optimistic (1) labeling training data
cursor = conn.execute("SELECT * from training_tweets WHERE Labels=1")
for row in cursor:
    print(row)
rows = cursor.fetchmany(10)
# Close the connection to the database


(343, 'Shout out to !!! I truly respect the way youre leading NY through this crisis!!! COVID19', '1')
(453, "I'll say it again, good work ! This is LEADERSHIP in the MEDIA!! MembersOfThePressWTF COVID19 VoteBlue2020", '1')
(581, 'Thank you so much for this UAE! indeed an answered prayer coronavirus Covid_19', '1')
(582, 'My mentor is out there fighting coronavirus in Detroit hospitals and I dont think Ive ever been prouder', '1')
(711, 'Three teams are fighting Covid19 day night: 1: Doctors, Nurses, Health Officials. 2: Modis PR team. 3: IT cell', '1')
(715, 'thanks to and for a brilliant zoom on covid19 emergency payment and tax and everything x', '1')
(1094, 'Thank you and G42! An impressive feat! Coronavirus diagnostics are ramped up in the UAE', '1')
(1281, 'When your congressman deploys with the to help fight the coronavirus . Thats leading by example.', '1')
(1356, 'Kylie Jenner to produce and donate hand sanitizer to hospitals battling covid19. Now thats an influencer.', '1')
(

In [38]:
# Retrieve data from the table
cursor = conn.execute("SELECT * from tweets")
rows = cursor.fetchmany(10)
rows

[('ᏉᎥ☻լꂅϮ',
  'astroworld',
  'wednesday addams as a disney princess keepin it [̲̅$̲̅(̲̅ιοο̲̅)̲̅$̲̅] 👸🏻💚♊️🧀🌵🌃🌙',
  '2017-05-26 05:46:42',
  '624',
  '950',
  '18775',
  'False',
  '2020-07-25 12:27:21',
  'If I smelled the scent of hand sanitizers today on someone in the past, I would think they were so intoxicated that… https://t.co/QZvYbrOgb0',
  '',
  'Twitter for iPhone',
  'False'),
 ('Tom Basile 🇺🇸',
  'New York, NY',
  'Husband, Father, Columnist & Commentator. Author of Tough Sell: Fighting the Media War in Iraq. Bush Admin Alum. Newsmax Contributor. Fmr Exec Dir NYSGOP',
  '2009-04-16 20:06:23',
  '2253',
  '1677',
  '24',
  'True',
  '2020-07-25 12:27:17',
  "Hey @Yankees @YankeesPR and @MLB - wouldn't it have made more sense to have the players pay their respects to the A… https://t.co/1QvW0zgyPu",
  '',
  'Twitter for Android',
  'False'),
 ('Time4fisticuffs',
  'Pewee Valley, KY',
  '#Christian #Catholic #Conservative #Reagan #Republican #Capitalist; Sports lover - #BBN #C

#7.   Perform exploratory data analysis (EDA) on the data to understand the distribution, trends, and relationships among variables.
#8.   Visualize the data using matplotlib, seaborn, or other visualization libraries.


In [32]:

conn.commit()
conn.close()

In [62]:


import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string
from wordcloud import WordCloud, STOPWORDS
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer